In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [6]:
# Create our features
X = df.drop(columns="loan_status")

X = pd.get_dummies(df)

# Create our target
y = df [["loan_status"]]

In [7]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,loan_status_low_risk,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.994958,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.070831,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,1.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,1.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [8]:
# Check the balance of our target values
y.value_counts()

loan_status
low_risk       68470
high_risk        347
dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [10]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_scaler = scaler.fit(X_train)
# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)


# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [11]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1) 
brf_model = brf_model.fit(X_train, y_train)
predictions = brf_model.predict(X_test)

In [12]:
# Calculated the balanced accuracy score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
acc_score = accuracy_score(y_test, predictions)

In [13]:
acc_score

0.9993606509735542

In [14]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)

In [15]:
cm

array([[  101,     0],
       [   11, 17093]])

In [16]:
# Print the imbalanced classification report
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,101,0
Actual 1,11,17093


In [17]:
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,101,0
Actual 1,11,17093


Accuracy Score : 0.9993606509735542
Classification Report
              precision    recall  f1-score   support

   high_risk       0.90      1.00      0.95       101
    low_risk       1.00      1.00      1.00     17104

    accuracy                           1.00     17205
   macro avg       0.95      1.00      0.97     17205
weighted avg       1.00      1.00      1.00     17205



In [18]:
# List the features sorted in descending order by feature importance
importances = brf_model.feature_importances_
importances


array([3.88875816e-03, 1.09722021e-02, 4.63029698e-03, 4.91052465e-03,
       4.32356764e-03, 1.14658138e-03, 1.97769275e-03, 3.07256400e-03,
       5.48703547e-04, 3.47683630e-03, 2.26412850e-03, 5.55713512e-03,
       6.41445446e-03, 3.83235797e-02, 2.98169244e-02, 3.17469706e-02,
       3.71951991e-02, 2.41160265e-03, 0.00000000e+00, 0.00000000e+00,
       3.42742594e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.08952319e-03, 4.23575822e-03, 1.70439714e-03, 2.63577610e-03,
       2.11715590e-03, 2.49833777e-03, 5.55730667e-03, 4.43709504e-03,
       4.27409570e-03, 1.29569137e-03, 1.42118037e-03, 4.70470383e-03,
       3.63053255e-03, 3.50021557e-03, 3.15620921e-03, 2.99437343e-03,
       3.57536110e-03, 4.79559763e-03, 5.37769294e-03, 4.67467484e-03,
       3.65961065e-03, 0.00000000e+00, 0.00000000e+00, 3.11822792e-03,
       4.47346673e-03, 2.66223399e-03, 3.27713114e-03, 1.30911189e-03,
       2.23941399e-03, 5.54693972e-03, 8.31160985e-04, 1.18127476e-03,
      

In [19]:
sorted(zip(brf_model.feature_importances_, X.columns), reverse=True)

[(0.32620489931446744, 'loan_status_high_risk'),
 (0.28266143696094514, 'loan_status_low_risk'),
 (0.03832357965844151, 'total_pymnt'),
 (0.037195199124358674, 'total_rec_int'),
 (0.03427425942688173, 'last_pymnt_amnt'),
 (0.031746970595770425, 'total_rec_prncp'),
 (0.02981692443571475, 'total_pymnt_inv'),
 (0.010972202088815648, 'int_rate'),
 (0.006562252554352294, 'issue_d_Jan-2019'),
 (0.006414454455600355, 'out_prncp_inv'),
 (0.0059385008919452835, 'total_bc_limit'),
 (0.005557306669106905, 'mths_since_rcnt_il'),
 (0.005557135117007859, 'out_prncp'),
 (0.005546939717230017, 'mths_since_recent_inq'),
 (0.005377692940576903, 'avg_cur_bal'),
 (0.005219652542070876, 'issue_d_Mar-2019'),
 (0.004910524654708113, 'annual_inc'),
 (0.0047955976258819265, 'acc_open_past_24mths'),
 (0.00470470383392367, 'max_bal_bc'),
 (0.004674674838549392, 'bc_open_to_buy'),
 (0.004630296977601816, 'installment'),
 (0.004473466728507074, 'mo_sin_old_rev_tl_op'),
 (0.00443709504077507, 'total_bal_il'),
 (0.0

### Easy Ensemble AdaBoost Classifier

In [20]:
X_train

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,loan_status_low_risk,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
3903,1600.0,0.0881,50.74,35964.0,24.12,0.0,0.0,20.0,0.0,1156.0,...,1,1,1,0,0,1,1,0,1,1
28390,9000.0,0.0756,280.21,41000.0,16.89,1.0,0.0,16.0,0.0,11157.0,...,1,1,0,1,0,1,1,0,1,1
15470,10000.0,0.1033,214.10,112000.0,17.75,0.0,0.0,14.0,0.0,16904.0,...,1,1,0,1,0,1,1,0,1,1
4279,36000.0,0.1033,1167.21,120000.0,19.95,0.0,1.0,13.0,0.0,63248.0,...,1,1,0,1,0,1,1,0,1,1
57514,18000.0,0.0881,570.81,51000.0,19.11,1.0,0.0,16.0,0.0,17586.0,...,1,1,0,1,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49100,8400.0,0.0646,257.30,37992.0,33.16,0.0,0.0,9.0,0.0,7316.0,...,1,1,0,1,1,0,1,0,1,1
20609,30000.0,0.0646,918.93,217000.0,19.56,0.0,2.0,25.0,0.0,142377.0,...,1,1,0,1,1,0,1,0,1,1
21440,20000.0,0.1180,442.88,35000.0,16.19,0.0,0.0,6.0,0.0,273.0,...,1,1,0,1,1,0,0,1,1,1
50057,11625.0,0.1447,273.34,30000.0,18.96,0.0,2.0,10.0,0.0,8237.0,...,1,1,0,1,1,0,1,0,1,1


In [21]:
y_train

,loan_status
3903,low_risk
28390,low_risk
15470,low_risk
4279,low_risk
57514,low_risk
...,...
49100,low_risk
20609,low_risk
21440,low_risk
50057,low_risk


In [22]:
from imblearn.ensemble import EasyEnsembleClassifier 

# X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(X, y,
#                                                     random_state=0)
eec = EasyEnsembleClassifier(n_estimators = 100, random_state=1)
eec.fit(X_train, y_train) 

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [25]:
y_pred = eec.predict(X_test)
print(confusion_matrix(y_test, y_pred))

[[  101     0]
 [    0 17104]]


In [26]:
# # Train the EasyEnsembleClassifier
# from imblearn.ensemble import EasyEnsembleClassifier
# from collections import Counter
# from sklearn.datasets import make_classification
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix

# #X, y = make_classification(n_classes=2, class_sep=2,
# #weights=[0.1, 0.9], n_informative=3, n_redundant=1, flip_y=0,
# #n_features=20, n_clusters_per_class=1, n_samples=1000, random_state=10)
# #print('Original dataset shape %s' % Counter(y))
# #X_train, X_test, y_train, y_test = train_test_split(X, y,
# #                                                    random_state=0)

# ez_model = EasyEnsembleClassifier(base_estimator=None, n_estimators=100, n_jobs=1, random_state=1, 
#                                    replacement=False, sampling_strategy='auto', verbose=0, 
#                                    warm_start=False)

# ez_model = ez_model.fit(X_train, y_train)


In [27]:
#ez_y_pred = ez_model.predict(X_test_scaled)

In [28]:
# Calculated the balanced accuracy score
acc_score = accuracy_score(y_test, predictions)

In [29]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

In [30]:
# Print the imbalanced classification report
# YOUR CODE HERE

In [31]:
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))


Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,101,0
Actual 1,11,17093


Accuracy Score : 0.9993606509735542
Classification Report
              precision    recall  f1-score   support

   high_risk       0.90      1.00      0.95       101
    low_risk       1.00      1.00      1.00     17104

    accuracy                           1.00     17205
   macro avg       0.95      1.00      0.97     17205
weighted avg       1.00      1.00      1.00     17205



In [32]:
#print (EasyEnsembleClassifier)

<class 'imblearn.ensemble._easy_ensemble.EasyEnsembleClassifier'>


In [ ]:
#pip list

In [ ]:
#print(imblearn.ensemble.__version__)